# Vitenskapelige beregninger
## Prosjekt I: Biofysikk 
##### Emil Spasov,  Alexander Umansky
##### 04.02.2023

In [1]:
#importing libraries

import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.animation import FuncAnimation
from scipy.optimize import curve_fit 
import datetime
from IPython.display import HTML
from scipy import ndimage

%matplotlib notebook

# Oppgave 1: Brownsk bevegelse


In [2]:
#Exercise 1
#function definitons

#---------------------------- Exercise 1b ------------------------------------#
def bm_1d(M,pr,dt,dx):
    '''
    Generates positional and time arrays for 1dimensional brownian motion

    Parameters
    ----------
    M : integer
        number of timesteps we are concidering.
    pr : float
        probability of the particle going right (isometry: pr = 0.5).
    dx : float 
        directional stepsize.
    dt : float
        time stepsize.

    Returns
    -------
    t : ndarray, size(M)
        array containg time.
    x : ndarray, size(M)
        array contiaining respetive position values on the x-axis.

    '''
    
    t = np.arange(0,M*dt,dt)
    x = np.zeros(M)
    rand = np.random.choice([-1,1],M,p=[1-pr,pr])
    x = np.cumsum(rand)
    
    return t,x

def bm_mult1(N,M,pr,dt,dx): 
    '''
    Generates positional and time arrays for 1dimensional brownian motion for multiple particles

    Parameters
    ----------
    N : integer
        number of particles.
    M : integer
        number of timesteps we are concidering.
    pr : float
        probability of the particle going right (isometry: pr = 0.5).
    dx : float 
        directional stepsize.
    dt : float
        time stepsize.

    Returns
    -------
    t : ndarray, size(M)
        array containg time.
    x : ndarray, size(M)
        array contiaining respetive position values on the x-axis.

    '''
    pos = np.zeros([N,M])
    t = np.arange(0,M*dt,dt)
    rand = np.random.uniform(0,1,[N,M])

    for i in range(0,N):
        for j in range(0,M):
            if rand[i][j]<=pr:
                pos[i][j]=pos[i][j-1]+dx
            else:
                pos[i][j]=pos[i][j-1]-dx

    return t, pos

def bm_mult2(N,M,pr,dt,dx):
    '''
    Generates positional and time arrays for 1dimensional brownian motion for multiple particles
    This is the second version feauturing improoved iteration method (cumulative sumation), making the runtime significalntly smaller

    Parameters
    ----------
    N : integer
        number of particles.
    M : integer
        number of timesteps we are concidering.
    pr : float
        probability of the particle going right (isometry: pr = 0.5).
    dx : float 
        directional stepsize.
    dt : float
        time stepsize.

    Returns
    -------
    t : ndarray, size(M)
        array containg time.
    x : ndarray, size(M)
        array contiaining respetive position values on the x-axis.

    '''
    pos = np.zeros([N,M])
    t = np.arange(0,M*dt,dt)
    rand = np.random.choice([-dx,dx],[N,M],p=[1-pr,pr])
    pos = np.cumsum(rand,axis=1)
    return t, pos


def empirical_var(t,x):
    '''
    Calculates empirical variance for the datasets as a function of time or amount of timesteps used.

    Parameters
    ----------
    t : ndarray, size(M)
        array containg time.
    x : ndarray, size(M)
        array contiaining respetive position values on the x-axis.

    Returns
    -------
    s : ndarray, size(M)
        array contianing variances for different sample sizes used.

    '''
    s = np.zeros(len(t))
    x_average = np.zeros(len(t))   
    
    for i in range(0,len(t)):
        res=0
        x_average[i] = np.average(x.T[i])
        res=np.sum((x.T[i]-x_average[i])**2)
        s[i]=(res/(len(x)-1))
    return s

def bm_2d(N,M,pu,pr,dt,dx):
    '''
    Generates positional and time arrays for 2dimensional brownian motion for multiple particles. The particle can move either in x or y direction per time step.

    Parameters
    ----------
    N : integer
        number of particles.
    M : integer
        number of timesteps we are concidering.
    pu : float
        probability of the particle going upwards (positive y directrion) (ismoetry: pu = 0.5).
    pr : float
        probability of the particle going right (isometry: pr = 0.5).
    dx : float 
        directional stepsize.
    dt : float
        time stepsize.
    Returns
    -------
    t : ndarray, size(M)
        array containg time.
    pos2D : ndarray, size(N,M,2)
        array contiaining respetive position values on the x and y-axis.

    '''
    t = np.arange(0,M*dt,dt);
    pos2D = np.zeros([2,N,M])
    rand_hv = np.random.choice([0,1],[N,M],p=[1-pu,pu]) 
    rand_ud = np.random.choice([-dx,dx],[N,M],p=[1-pr,pr]) 
    rand_lr = np.random.choice([-dx,dx],[N,M],p=[1-pr,pr])
    rand_lr[rand_hv==1]=0
    rand_ud[rand_hv==0]=0
    pos_hv = np.cumsum(rand_ud, axis = 1)
    pos_lr = np.cumsum(rand_lr, axis = 1)
    pos2D[0]=pos_lr
    pos2D[1]=pos_hv

    return t, np.transpose(pos2D,(1,2,0))

def animate_particle(t,pos2D,N,M,pu,pr,lim,colors,dt,dx):
    '''
    Animates particle 2d-spacial movement for multiple particles.

    Parameters
    ----------
    t : ndarray, size(M)
        array containg time.
    pos2D : ndarray, size(N,M,2)
        array contiaining respetive position values on the x and y-axis.
    dt : float
        time stepsize.
    dx : float 
        directional stepsize.
    N : integer
        number of particles.
    M : integer
        number of timesteps we are concidering.
    pu : float
        probability of the particle going upwards (positive y directrion) (ismoetry: pu = 0.5).
    pr : float
        probability of the particle going right (isometry: pr = 0.5).
    lim : integer
        Animation widow dimensions.
    colors : array of strings
        Specification of clours we use for different particles in the animation.

    Returns
    -------
    HTML(ani.to_jshtml()) : Python.display.HTML object
        The animation object

    '''
    fig, ax = plt.subplots()
    graph = []
    fig.suptitle(f'Brownian motion of particles with $p_R = {pr}$ and $p_U = {pu}$')
    ax.set_ylim(-lim,lim)
    ax.set_xlim(-lim,lim)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    
    for i in range(0,len(pos2D)):
        x=pos2D[i].T[0]
        y=pos2D[i].T[1]
        graph.append(ax.plot(x[0],y[0],marker='o',color=colors[i]))

    def animation_frame(i):
        for k in range(0,len(pos2D)):
            data= (pos2D[k].T[0][i*10],pos2D[k].T[1][i*10])
            graph[k][0].set_data(data)
        
        return graph,
        
    ani = FuncAnimation(fig, func = animation_frame, frames=int(M/10), interval=dt*100)
    plt.show()
    return HTML(ani.to_jshtml())


#---------------------------- Exercise 1i ------------------------------------#

def origo(pos, N, M, x=0,y=0):
    '''
    Studies a spesified point on the grid, and registeres number of times particle(s) return to it during a given time interval.

    Parameters
    ----------
    pos : ndarray, size (N,2)
        Coordinates to the position we observe.
    N : integer
        number of particles.
    M : integer
        number of timesteps we are concidering.

    Returns
    -------
    n : integer
        Number of times particle returned to the spesified cell.

    '''
    n = np.zeros(M)
    pkt=[x,y]
    for i in range(0,N):    
        temp=pos.T[i]
        for j in range(1,M):
            if np.all(temp[j]==pkt):
                n[j]=n[j-1]+1
            else:
                n[j]=n[j-1]
    return n



### Oppgave 1a
Difusjon i en dimensjon følger likningen:
$$ \frac{\partial  \varphi  \big(x, t\big)}{\partial t} = D  \frac{\partial^2 \varphi  \big(x, t\big)}{\partial x^2} \tag{1} $$
hvor D er en material avhengig konstant. Antar at partikkeltettheten $\varphi$ er en normal sansynlightesfordeling med forventningsverdi  $\mu = 0$, og varians $ \sigma ^{2} = at$. Partikkeltetthet vil da kunne skrives som:  $$ \varphi  \big(x, t\big) = \frac{1}{ \sqrt{2  \pi a t} } exp \big\{- \frac{1}{2}  \frac{ x^{2} }{at} \big\} \tag{2} $$ Partiellderiverer (2) en gang med hennsyn på tid og to ganger med hennsyn på posisjon som gir oss ligningen:
$$ \frac{x^2}{2at^{5/2}} - \frac{1}{2t^{3/2}} = D\frac{x^2}{a^2t^{5/2}} - D\frac{1}{at^{3/2}} \tag{3}$$
Da får vi 2 ligninger
$$ \frac{1}{2} = D  \;\;\;\;\;\;\;;\;\;\;\;\;\; \frac{1}{2a} = D \frac{1}{2  a^{2} }  $$ det eneste verdi for a som løser både likning at $a = 1$


### Oppgave 1c



In [3]:
dt = 1
dx = 1

#---------------------------- Exercise 1c ------------------------------------#

pr=[0.45,0.5,0.55]
M = 10000
t,x = bm_1d(M, pr[0],dt,dx)

plt.figure()
plt.plot(t,x, label = f"$p_R = {pr[0]}$")

t,x = bm_1d(M, pr[1],dt,dx)
plt.plot(t,x, label = f"$p_R = {pr[1]}$")

t,x = bm_1d(M, pr[2],dt,dx)
plt.plot(t,x, label = f"$p_R = {pr[2]}$")

plt.title("Brownian motion in 1D")
plt.xlabel("Time [s]")
plt.ylabel("Left / Right")
plt.legend()
 

<IPython.core.display.Javascript object>

**Svar:** Ved å velge stor M blir simuleringen vår mer statistisk representativ og kan avgi mulige svakheter om modellen hadde hatt feil. Orange grafen (isometri) til tross for tilfeldige fluktasjoner holder seg på et kosntant nivå selv etter 10 000 sekkunder. Endrer vi sansynlighet selv med 0.05 er denne forksjellen godt synelig på den grafiske visualiseringen etter lang tid.


### Oppgave 1d

In [4]:
#--------------------------- Exercise 1d -------------------------------------#
N=1000
M=1000

start_time = datetime.datetime.now()

t,x = bm_mult1(N, M, 0.5,dt,dx)

end_time = datetime.datetime.now()

plt.figure()
plt.plot(t,x[0], label = "Particle 1")
plt.plot(t,x[1], label = "Particle 2")
plt.plot(t,x[2], label = "Particle 3")

plt.title("Brownian motion of multiple particles in 1D")
plt.xlabel("Time [s]")
plt.ylabel("Left / Right")
plt.legend()

print("Time used with first method (Exercise 1d.)",end_time - start_time)



<IPython.core.display.Javascript object>

Time used with first method (Exercise 1d.) 0:00:01.243805


### Oppgave 1e

In [5]:
#--------------------------- Exercise 1e -------------------------------------#

start_time = datetime.datetime.now()

t,x = bm_mult2(N, M, 0.5,dt,dx)
end_time = datetime.datetime.now()

plt.figure();
plt.plot(t,x[0], label = "Particle 1");
plt.plot(t,x[1], label = "Particle 2");
plt.plot(t,x[2], label = "Particle 3");

plt.title("Brownian motion of multiple particles in 1D")
plt.xlabel("Time [s]")
plt.ylabel("Left / Right")
plt.legend()
    
print("Time used with improved method (Exercise 1e.) ",end_time - start_time)



<IPython.core.display.Javascript object>

Time used with improved method (Exercise 1e.)  0:00:00.052979


**Svar:** I oppgave 1d brukte vi for-løkker og if-setninger fordi det er nærmest naturlig tankegang og er lett å lese og forstå, men er ikke effektiv når vi velger store antall partikler og tidsteg. Derfor i oppgave 1e brukte vi np.cumsum og array manipulasjon og adisjon. På den måten får vi 10-20 ganger mindre kjøretid. Grunnet til dette er at numpy biblioteket er skrevet i C++. C++ er lav nivå programeringsspråk og er derfor mye raskere. 

### Oppgave 1f

In [6]:
#--------------------------- Exercise 1f -------------------------------------#

def f(x,a,b):
    return a * x + b;

popt, pcov = curve_fit(f, t, x[0])

plt.figure();
plt.plot(t,x[0], label = "$Motion$");
plt.plot(t,f(t,popt[0],popt[1]), label = "Linear fit")    
plt.title("Brownian motion of particle in 1D with linear fit")
plt.xlabel("Time [s]")
plt.ylabel("Left / Right")
plt.legend()

plt.figure();
plt.plot(t,empirical_var(t, x), label = "Empirical variance");
plt.plot(t,t,label="Variance (calculated in 1a)")
plt.title("Variance")
plt.xlabel("Time [s]")
plt.ylabel("Variance $\sigma^2$")
plt.legend()



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Svar:** Fra grafen over ser vi at variansen er en lineær funksjon av tid med proporsjonalitetskosntant $ a \approx 1$. Oransje grafen betegner variansen $\sigma^2 = at$ med $a = 1$, er variansen til en analytisk løsning. Approksimasjonen blir mer og mer nøyaktig med stort N fordi da minimeres det tilfeldig støy. I våres tilfellet har vi brukt $N=1000$ dette fører til resultatet sammenfaller ganske nøyaktig med det vi fant i 1a som vist i figuren. Jo mer vi øker N jo nærmere blir numersiske tilnærmingen til den analytiske løsningen. 

### Oppgave 1g: Isotrop system

In [7]:
#---------------------------- Exercise 1g ------------------------------------#
colors = ['red','green','blue', 'orange']

N=4
M=100

pu=0.5
pr=0.5

t,pos2D = bm_2d(N,M,pu,pr,dt,dx)
plt.figure()
plt.title(f"Brownian motion of {N} particles in 2D")
plt.xlabel("x")
plt.ylabel("y")
for i in range(0,N):
    plt.plot(pos2D[i].T[0],pos2D[i].T[1],color=colors[i])
    plt.plot(pos2D[i][-1][0],pos2D[i][-1][1],marker='o',color=colors[i])

animate_particle(t,pos2D, N, M, pu, pr, 60,colors,dt,dx)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Oppgave 1g: Anisotrop system

In [8]:
dt=1
dx=1
N=4
M=100

pu=0.2
pr=0.8
t,pos2D = bm_2d(N,M,pu,pr,dt,dx)

plt.figure()
plt.title(f"Brownian motion of {N} particles in 2D")
plt.xlabel("x")
plt.ylabel("y")
for i in range(0,N):
    plt.plot(pos2D[i].T[0],pos2D[i].T[1],color=colors[i])
    plt.plot(pos2D[i][-1][0],pos2D[i][-1][1],marker='o',color=colors[i])

animate_particle(t,pos2D, N, M, pu, pr, 60,colors,dt,dx)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Svar:** Som forventet for et isotrop system observer vi ingen tydelig bevegelse i noen retning. Pariklene har ingen tydelig eller regelmessig bevegelsesmønster bort fra startlokasjonen. Om vi endrer $p_r$ og $p_u$ til andre verdier enn 0.5 blir migrasjonsmønstet tydeligere. I ekesemplet over bruker vi $p_r = 0.8$ og $p_u = 0.2$ som betyr at vi ser oftest bevegelse i x-retning og som oftest mot høyre.

# Oppgave 1h
Viktig anntakelse: Isometri!$$ $$
$P(x=0, t=1)$ vil være $0$ uansett dimensjon. Du gjør et tidssteg unna origin. Det minste antallet steg som trengs for å kunne ha $n > 0$ er $2$ steg. $$ $$
$P(x=0, t=2)$ er nå ikke lenger $0$, men sansynligheten vil variere etter dimensjonene og tiltatte bevegelser. 
I $1D$ er det kunn $2$ tiltatte retninger (Høyre / Venstre) så for at $n=1$ må steg $2$ være det omvendte av steg 1: enten høyre - venstre, eller venstre- høyre. Sansynligheten for hvær altærnativ er $0.5  \cdot 0.5$ og med 2 altærnativer er $P = 2 \cdot0.5 \cdot0.5 = 0.5$ $$ $$
I $2D$ er det nå 4 tiltatte retninger (Øst, Vest, Nord, Sør). Hvis første steg var nordover må andre steg være sør som utgjør en fjærdel. Vi har 4 mulige altærnativer hver med $0.25 \cdot0.25$ sansynlighet: $P = 4 \cdot0.25 \cdot0.25 = 0.25$

### Opgave 1i

In [9]:
#---------------------------- Exercise 1i ------------------------------------#

start_time = datetime.datetime.now()
dx=1
dt=1
N=20
M=8000
pu = 0.5
pr = 0.5
plt.figure()
plt.title('Number of times a particle passes point [0,0] ')
t,pos = bm_mult2(N,M,pr,dt,dx)
n = origo(pos.T,N,M)
plt.plot(t,n,'b',label="1D motion")

t,pos2d = bm_2d(N,M,pu,pr,dt,dx)
n2d = origo(pos2d.T,N,M)
plt.plot(t,n2d,'r',label="2D motion")

plt.xlabel("Time [s]")
plt.ylabel("Number of times passed origo")
plt.legend()

print(f"Total passes 1D: {n[-1]}")
print(f"Total passes 2D: {n2d[-1]}")

end_time = datetime.datetime.now()
print("Time used (Exercise 1i.)",end_time - start_time)

<IPython.core.display.Javascript object>

Total passes 1D: 11.0
Total passes 2D: 1.0
Time used (Exercise 1i.) 0:00:04.332484


**Svar:** I grafen ser vi først og fremst at i 1D tilfellet returnerer partiklene mye oftere til origo enn i 2D. Dette skilles at modellen lar partikkelen gå i kun én retning om gangen (vertikalt eller horisontalt). Dette gjør det vanskeligere for en allerede avvikende partikkel å komme tilbake i få steg. Grafisk er det ingen stigning på de intervallene. Den sørste økingen skjer derfor også i starten av all bevegelse og går saktere mot slutten. Som sagt i oppgave 1h blir det mindre sansynlig for partikkelen å returnere når vi introduserer flere dimensjoner og derfor også mulige bevegelsesretninger. For 8 000 sekunder er det nesten ingen registrerte returneringer. Hvis en partikkel i 2d først kommer seg unna origo blir det mye lettere for den å gå seg bort for alltid. ("A drunk man will find his way home, but a drunk bird may get lost forrever" -Shizuo Kakutani).  

In [10]:
#Exercise 2
#Function definitions

def ind(pos,steps,L):
    '''
    Calculates the index of a given position in a numpy mesh grid with
    dimensions [steps,steps] 

    Parameters
    ----------
    pos : ndarray,shape(2,)
        meshgrid position
    steps : int
        discretization of the meshgrid
    L : float
        Limits of the grid.
    Returns
    -------
        int
        index of the point in the meshgrid.

    '''
    
    stepsize = (2*L/steps)
    return int((pos+L)/stepsize)


def create_tumors(xv,yv,t_area,t_centers,t_coeff,n,dx):
    '''
    Creates a meshgrid with zones with different stepsize (tumors)

    Parameters
    ----------
    xv : ndarray
        meshgrid x.
    yv : ndarray
        meshgrid y.
    t_area : float
        area of the tumors.
    t_centers : ndarray, shape(n,2)
        positions of the centers of the tumor.
    n : int
        number of tumors.

    dx : float
        default value of dx outside of tumors


    Returns
    -------
    dx_arr : ndarray, same shape as xv and yv
        meshgrid with dx values.

    '''
    dx_arr=(xv**0+yv**0)*dx/2

    t_radius = np.sqrt(t_area*10/np.pi)
    inside = []
    for i in range(0,n):
        inside.append(np.sqrt((xv-t_centers[i][0])**2+(yv-t_centers[i][1])**2)<=t_radius)
        dx_arr[inside[i]]*=np.sqrt(t_coeff[i])
    return dx_arr


def bm_step(pos,xv,yv,N,pu,pr,dt,dx0,dx_arr,steps,L): 
    '''
    Calculates a step the brownian motion for N particles 

    Parameters
    ----------
    pos : ndarray, shape(N,2)
        initial positions of the particles.
    xv : ndarray, shape(steps,steps)
        meshgrid x.
    yv : ndarray, shape(steps,steps)
        meshgrid y.
    N : int
        number of particles.
    pu : float [0,1]
        probability to go vertical (vs. horisontal).
    pr : float [0,1]
        probablity to go right(up if vertical).
    dt : float
        timestep
    dx0 : float
        stepsize, default outside tumors
    dx_arr : ndarray(steps,steps)
        stepsizes array.
    steps : int
        discretization constant of the meshgrid.
    L : float
        limits of the work area.

    Returns
    -------
    pos : ndarray(N,2)
        Positions array for N number of particles.
    hits : int
        number of times a particle have been inside a tumor .

    '''
    rand_hv = np.random.choice([0,1],[N], p=[1-pu,pu]) 
    rand_ud = np.random.choice([-1,1],[N], p=[1-pr,pr]) 
    rand_lr = np.random.choice([-1,1],[N], p=[1-pr,pr])
    mod_ud=np.ones(N)
    mod_lr=np.ones(N)
    hits = []
    for i in np.arange(0,N):

        if rand_hv[i]==1:
            rand_lr[i]=0
        if rand_hv[i]==0:
            rand_ud[i]=0
        mod_ud[i] = rand_ud[i]*dx_arr[ind(pos[i][0],steps,L)][ind(pos[i][1],steps,L)]
        mod_lr[i] = rand_lr[i]*dx_arr[ind(pos[i][0],steps,L)][ind(pos[i][1],steps,L)]
        if abs(pos[i][0]+mod_lr[i])>=abs(L):
            mod_lr*=0
            mod_ud*=0

        if abs(pos[i][1]+mod_ud[i])>=abs(L):
            mod_lr*=0
            mod_ud*=0
        if dx_arr[ind(pos[i][0],steps,L)][ind(pos[i][1],steps,L)] != dx0:
            hits.append([pos[i][0],pos[i][1]])
            
    pos_change=np.array([mod_lr,mod_ud])
    pos=pos + pos_change.T

    return pos,hits

def bm_different_dx(M,N,xv,yv,pu,pr,dt,dx,dx0,steps,L,):
    '''
    calculates the brownian motion for N particles and M timesteps

    Parameters
    ----------
    M : int
        timesteps.
    N : int
        number of particles.
    xv : ndarray, shape(steps,steps)
        meshgrid x.
    yv : ndarray, shape(steps,steps)
        meshgrid y.
    N : int
        number of particles.
    pu : float [0,1]
        probability to go vertical (vs. horisontal).
    pr : float [0,1]
        probablity to go right(up if vertical).
    dt : float
        timestep
    dx0 : float
        stepsize, default outside tumors
    dx : ndarray(steps,steps)
        stepsizes array.
    steps : int
        discretization constant of the meshgrid.
    L : int
        limits of the work area.

    Returns
    -------
    t : ndarray, shape(M,)
        array with times.
    ndarray, shape(N,M,2)
        path of the particles.
    ndarray, shape(h,2), where h is the number of hits
        ndarray with the positions of where the particles have been in a tumor.

    '''
    
    hits=[]
    t = np.arange(0,M*dt,dt);
    
    pos = np.random.uniform(-L,L,[M,N,2])
    
    if N==4:
        pos0 = [[-1/2*L,-1/2*L],[-1/2*L,1/2*L],[1/2*L,-1/2*L],[1/2*L,1/2*L]]
        pos[::][0]+=pos0
    
    for i in np.arange(1,M):
        pos[i], h = bm_step(pos[i-1],xv,yv,N,pu,pr,dt,dx0,dx,steps,L)      
        hits.extend(h)
    return t, np.transpose(pos,(1,0,2)),np.array([hits])[0].T

#----------------------------Oppgave 2e-----------------------------------#

def intensity (pos,xmin,xmax,ymin,ymax,nx,ny):
    '''
    Calculates the intensity of individual particle motion

    Parameters
    ----------
    pos : ndarray
        path of the particle.
    xmin : float
        lower limit of x.
    xmax : float
        upper limit of x.
    ymin : float
        lower limit of y.
    ymax : float
        upper limit of y.
    nx : int
        number of elements in the x direction.
    ny : int
        number of elements in the y direction.

    Returns
    -------
    I : ndarray
        matrix with intensity values of individual particle.
    xedges : ndarray, shape(nx+1,)
        The bin edges along the first dimension.

    yedges : ndarray, shape(ny+1,)
        The bin edges along the second dimension.

    '''
    I, xedges, yedges = np.histogram2d(pos[0],pos[1],bins=(nx,ny),range=([xmin, xmax], [ymin, ymax]))
    return I,xedges, yedges

def intensity_all(pos_arr,xmin,xmax,ymin,ymax,nx,ny,N):
    '''
    Sums over the intensity of N number particles

    Parameters
    ----------
    pos_arr : ndarray
        position vector of pos_arr.
    xmin : float
        lower limit of x.
    xmax : float
        upper limit of x.
    ymin : float
        lower limit of y.
    ymax : float
        upper limit of y.
    nx : int
        number of elements in the x direction.
    ny : int
        number of elements in the y direction.
    N : int
        number of particles.

    Returns
    -------
    I : ndarray, shape(n,m)
        Comulative intensity of all particles motion.
    xedges : ndarray, shape(nx+1,)
        The bin edges along the first dimension.
    yedges : ndarray, shape(ny+1,)
        The bin edges along the second dimension.

    '''
    I, xedges, yedges = intensity(pos_arr[0].T,xmin,xmax,ymin,ymax,nx,ny)
    for i in range (1,N):
        H,xedges,yedges=intensity(pos_arr[i].T,xmin,xmax,ymin,ymax,nx,ny)
        I+=H
    return I, xedges, yedges


def Sobel(n,m,I):
    '''
    Applies a Sobel filter on a Intensity matrix

    Parameters
    ----------
    n : int
        number of elements in x direction.
    m : int
        number of elements in y direction.
    I : ndarray, shape (n,m)
        Intensity.

    Returns
    -------
    S : ndarray, shape(n,m)
        Intensity with applied Sobel filter.
    '''
    g_x = np.matrix("1 0 -1; 2 0 -2; 1 0 -1")
    g_y = np.matrix("1 2 1; 0 0 0; -1 -2 -1")

    X = np.zeros((n-2, m-2))
    Y = np.zeros((n-2, m-2))
    S = np.zeros((n-2, m-2))

    for i in range(2, n-1):
        for j in range(2, m-1):
            A_local = np.array([[I[i-1][j-1], I[i-1][j], I[i-1][j+1]], [I[i][j-1], I[i][j], I[i][j+1]], [I[i+1][j-1], I[i+1][j], I[i+1][j+1]]])
           
            X[i-1][j-1] = np.sum(np.multiply(g_x, A_local))
            Y[i-1][j-1] = np.sum(np.multiply(g_y, A_local))
            S[i-1][j-1] = np.sqrt((X[i-1][j-1])**2 + (Y[i-1][j-1])**2)
    
    #normalization of X, Y and S
    X = X / np.max(X)
    Y = Y / np.max(Y)
    S = S / np.max(S)
    
    return S


# Oppgave 2a
Difusjonskonstanen D brukt i likning (1) bestemes på følgende måte:
$$D = \frac{   \Delta x^{2} }{ 2\Delta t}$$
vi antar $\Delta x = 0.4 \mu m $, $\Delta t = 0.01 s $ i et friskt vev, det gir en difusjonskonstant $D = 16 \mu m^2 s^{-1}$  
$$ $$ Om vi introduserer $n$ antall tumorer med koeffisienter $ \kappa_{n} $ vil de redusere $\Delta x$ og dermed Disfusjonskonstanten på følgende måte: 
$$ D = \frac{ \big(\Delta x- \sqrt{ \prod_{n}   \kappa _{n}  }\big) ^{2} }{ 2\Delta t} $$

### Oppgave 2c

In [11]:
dx0 = 0.4  #[micrometer]
dt = 0.01  #[s]

n = 15     #number of tumors
L = 20     #[micrometer]
steps = 1000
stepsize = 2*L/steps

N=2
M=1000

pu=0.5
pr=0.5

x =np.linspace(-L,L,steps)
y =np.linspace(-L,L,steps)
xv,yv = np.meshgrid(x,y,indexing='xy')
t_area = dx0**2*np.pi #area of tumor
t_centers = np.random.uniform(-L+np.sqrt(t_area/np.pi), L-np.sqrt(t_area/np.pi),(n,2))
t_coeff = np.ones(n)*0.1

dx_arr = create_tumors(xv, yv, t_area, t_centers,t_coeff, n, dx0)
t,pos,hits= bm_different_dx(M,N,xv,yv,pu,pr,dt,dx_arr,dx0,steps,L)

#PLOTS
plt.figure()

plt.contourf(xv,yv,dx_arr, cmap='binary')
if hits.size>0:
    plt.plot(hits[1],hits[0], marker='o',color='r',linestyle= 'None', markersize = 0.1)
    plt.plot([],[],marker='o',color='r',linestyle= 'None',label=f'Tumor hits $n={hits[0].size}$') #just for the legend
    plt.legend()

plt.title("Tumor positions", fontsize=24 )
plt.xlabel('x $[\mu m]$')
plt.ylabel('y $[\mu m]$')

plt.figure()
plt.title("Particles path", fontsize=24 )
plt.xlim(-L,L)
plt.ylim(-L,L)
for i in range(0,N):
    plt.plot(pos[i].T[1],pos[i].T[0])
plt.xlabel('x $[\mu m]$')
plt.ylabel('y $[\mu m]$')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Text(0, 0.5, 'y $[\\mu m]$')

**Svar:** Figurene over viser samtidig posisjonene til tumorene og bevegelsesmønsteret til 2 virrevandrere. De røde prikkene betegner stedene hvor virrevandrere ble påvirket av tumorer. Med kun 2 partikler og ikke store tumor arealer kan det være vanskelig å se klart effekt av overlapping på dx (kan avhenge av "fall of the cards"). Det vi kan likevel se klart, er at virrevandrere beveger mye kortere og tettere i noen "clusters". Disse stedene sammenfaller med tumorenes posisjoner. 

### Oppgave 2d

**Svar:** For å nærme vår simulering best mulig til virkeligheten vil vi nå introdusere en modelering av "harde vegger" som en fysisk begrensning på difusjonsbevegelsene. Dersom en virrevandrer kommer i kontakt med grid-edges avbryter vi bevegelsen som fører partikkel ut, og rekalkulerer ny bevegelse. Parikkler kan derfor aldri komme ut av området. Dette er uønskelig siden intenting hindrer partikklene å helt forsvinne fra området vi fokuserer på i analysen. Svakheten ved denne modellen er at i virkeligheten kan partikler strømme både inn og ut av interesseområdet. Hvis vi likevel anntar et veldig høyt antall partikler kan fluksen annsen i balanse mellom innvandrere og utvandre og totalt antall partikler i systemet er dermed konstant. En annen ting vi endret var å introdusere flere forksjellige startposisjoner til virrevandre for å sikre en mye mer reaslistisk fordeling og bevegelse innenfor vårt område. 

### Oppgave 2f og 2g

In [12]:
dx0 = 0.4  #[micrometer]
dt = 0.01  #[s]

n = 25    #number of tumors
L = 20     #[micrometer]
steps = 1000
stepsize = 2*L/steps

N=40
M=1000

pu=0.5
pr=0.5

m = 40 # number of bins

x =np.linspace(-L,L,steps)
y =np.linspace(-L,L,steps)
xv,yv = np.meshgrid(x,y,indexing='xy')
t_area = dx0**2*np.pi #area of tumor
t_centers = np.random.uniform(-L+np.sqrt(t_area/np.pi), L-np.sqrt(t_area/np.pi),(n,2))
t_coeff = np.random.uniform(0.3,0.45,[n])


dx_arr = create_tumors(xv, yv, t_area, t_centers,t_coeff, n, dx0)
t,pos,hits= bm_different_dx(M,N,xv,yv,pu,pr,dt,dx_arr,dx0,steps,L)

I,xedges,yedges=intensity_all(pos,-L,L,-L,L,m,m,N)

I=I/(M*N)

sobel = Sobel(m,m,I)
sobel_sp = ndimage.sobel(I)
    
#PLOTS
plt.figure()
plt.contourf(xv,yv,dx_arr, cmap='binary')
if hits.size>0:
    plt.plot(hits[1],hits[0], marker='o',color='r',linestyle= 'None', markersize = 0.1)
plt.plot([],[],marker='o',color='r',linestyle= 'None',label=f'Tumor hits $n={hits[0].size}$') #just for the legend
plt.title("Tumor positions", fontsize=24 )
plt.xlabel('x $[\mu m]$')
plt.ylabel('y $[\mu m]$')
plt.legend()

extent = [xedges[0], xedges[-1], yedges[-1], yedges[0]]

fig = plt.figure()
fig.suptitle("Intensity",fontsize=24)
fig.tight_layout()

ax1 = fig.add_subplot(111)

ax1.imshow(I, extent=extent,cmap='gray')
ax1.invert_yaxis()

ax1.set_xlabel('x $[\mu m]$')
ax1.set_ylabel('y $[\mu m]$')
#-------------------------------------------------------------------------

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Text(0, 0.5, 'y $[\\mu m]$')

In [13]:
m = 100

I,xedges,yedges=intensity_all(pos,-L,L,-L,L,m,m,N)

I=I/(M*N)

start_time = datetime.datetime.now()
sobel = Sobel(m,m,I)
end_time = datetime.datetime.now()
print(f"Time using algorithm 2: {end_time-start_time}")

start_time = datetime.datetime.now()
sobel_sp = ndimage.sobel(I)
end_time = datetime.datetime.now()
print(f"Time using scipy.ndimage.sobel: {end_time-start_time}")

#PLOTS
plt.figure()
plt.contourf(xv,yv,dx_arr, cmap='binary')
if hits.size>0:
    plt.plot(hits[1],hits[0], marker='o',color='r',linestyle= 'None', markersize = 0.1)
plt.plot([],[],marker='o',color='r',linestyle= 'None',label=f'Tumor hits $n={hits[0].size}$') #just for the legend
plt.title("Tumor positions", fontsize=24 )
plt.xlabel('x $[\mu m]$')
plt.ylabel('y $[\mu m]$')
plt.legend()

extent = [xedges[0], xedges[-1], yedges[-1], yedges[0]]

fig = plt.figure()
fig.suptitle("Intensity",fontsize=24)
fig.tight_layout()

ax = fig.add_subplot(111)    # The big subplot
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.set_title("Unfiltered")
ax1.imshow(I, extent=extent,cmap='gray')
ax1.invert_yaxis()

ax2.set_title("Sobel filter")
ax2.imshow(sobel, extent=extent,cmap='gray')
ax2.invert_yaxis()


# Turn off axis lines and ticks of the big subplot
ax.spines['top'].set_color('none')
ax.spines['bottom'].set_color('none')
ax.spines['left'].set_color('none')
ax.spines['right'].set_color('none')
ax.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)

ax.set_xlabel('x $[\mu m]$')
ax.set_ylabel('y $[\mu m]$')
#-------------------------------------------------------------------------
plt.figure()
plt.title("Intensity with Sobel filter from SciPy")
plt.imshow(sobel_sp, extent=extent,cmap='gray')
plt.xlabel('x $[\mu m]$')
plt.ylabel('y $[\mu m]$')
plt.gca().invert_yaxis()

Time using algorithm 2: 0:00:00.529425
Time using scipy.ndimage.sobel: 0:00:00.001000


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Diskusjon:** Ved sammeligning av tumor-mappet mot intentistets plottet ser vi at de setmmer godt overens. Ut fra bildet med intensitet kan vi allerede registrere en stor anndel av tumorene. Ligger vi til et sobelfilter ser vi fortettningene enda tydeligere og arealet til tumorene begynner å bli godt gjenkjennelig. Hvis vi velger for stor antall "bins" får vi uskarp bilde uten noe mønster. Grunnet til dette er at hvis vi velger for små "bins" er det fære partikler som går gjennom eksakte posisjonen og da blir intensiteten lav. 
Vi brukte to Sobel filter algoritmer - første er algoritme 2 og andre er fra SciPy biblioteket. Vi merker at scipy gjør det mye raskere, men våres algoritme produserer et litt tydeligere bilde.